In [12]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from requests.auth import HTTPBasicAuth
import random 
import time
import math

In [13]:
consumer_key = 'Lq3V1hAxII2Fv8YSdcrhfA538HzhsfzhelYqSUgzGU0A8AEU'
consumer_secret_key = 'jHJjRSzVQBjCizDYmFg4cqWza2R8TOCcLaecRbXrJQcseLu8Lr0AqYXL1ldkAx8A'

# EPO OAuth2 token URL
token_url = "https://ops.epo.org/3.2/auth/accesstoken"

def get_token():
    # Request access token
    response = requests.post(
        token_url,
        auth=HTTPBasicAuth(consumer_key, consumer_secret_key),
        data={"grant_type": "client_credentials"},
    )
    
    # Extract token
    if response.status_code == 200:
        token = response.json()["access_token"]
        return token
    else:
        print("Failed to retrieve access token.")
        return None

In [14]:
#patent search without countries for faster collection (used up until 1866)
def patent_search(date_range, token, start, end):

    search_url = "https://ops.epo.org/rest-services/published-data/search"

    # Headers with authentication
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/xml",
        "User-Agent": "ProjectAcademicBot/1.0 (for educational purposes only, contact: collen.ellis@sorbonne-universite.fr)",
        "Connection": "keep-alive"
    }
    
    params = {
    "q": f'pd={date_range}',
    "range": f"{start}-{end}",
    }
    
    response = requests.get(search_url, headers=headers, params=params)
    
    if response.status_code == 200:
        
        patents_list = []
        
        # Namespace dictionary (important for parsing)
        ns = {"epo": "http://www.epo.org/exchange"}
        
        xml_data = response.text
        root = ET.fromstring(xml_data)
        
        for patent in root.findall(".//epo:document-id", namespaces=ns):
            country = patent.find("epo:country", namespaces=ns)
            doc_number = patent.find("epo:doc-number", namespaces=ns)
        
            if country is not None and doc_number is not None:
                patents_list.append({
                "country": country.text,
                "doc_number": doc_number.text
            })
        
        return patents_list
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []

In [20]:
amount_per_year = pd.read_csv('uncollected_years_amount.csv')
# Drop all rows where year is between 1928 and 1965 (inclusive)
amount_per_year = amount_per_year[~amount_per_year["year"].between(1928, 1983)]
display(amount_per_year)

,year,sample_amount
55,1984,1714
56,1985,1732
57,1986,1750
58,1987,1767
59,1988,1785
60,1989,1803
61,1990,1821
62,1991,1839
63,1992,1857
64,1993,1875


In [27]:
# Monthly date suffixes
months = {
    "01": ("0101", "0131"),
    "02": ("0201", "0228"),  # Not handling leap years
    "03": ("0301", "0331"),
    "04": ("0401", "0430"),
    "05": ("0501", "0531"),
    "06": ("0601", "0630"),
    "07": ("0701", "0731"),
    "08": ("0801", "0831"),
    "09": ("0901", "0930"),
    "10": ("1001", "1031"),
    "11": ("1101", "1130"),
    "12": ("1201", "1231"),
}

range_size = 100
max_total = 2000  # EPO API cap
all_patents = []

# Loop over each year/amount in your DataFrame
for _, row in amount_per_year.iterrows():
    year = int(row["year"])
    total_amount = int(row["sample_amount"])

    token = get_token()
    year_results = []

    monthly_quota = math.floor(total_amount / 12)
    remaining = total_amount

    for i, (month, (start_suffix, end_suffix)) in enumerate(months.items()):
        if remaining <= 0:
            break

        # Ensure the last month makes up for any rounding loss
        target_this_month = min(monthly_quota, remaining)
        if i == 11:
            target_this_month = remaining

        collected_this_month = []
        start = 1
        end = range_size
        date_range = f"{year}{start_suffix}-{year}{end_suffix}"

        while len(collected_this_month) < target_this_month:
            time.sleep(random.uniform(4, 4.5))
            results = patent_search(date_range, token, start, end)

            if results is None or len(results) == 0:
                print(f"Failed for {year}-{month}, start={start}")
                break

            collected_this_month.extend(results)
            if len(results) < range_size:
                break

            start += range_size
            end += range_size

        collected_this_month = collected_this_month[:target_this_month]  # trim extras
        year_results.extend(collected_this_month)
        remaining -= len(collected_this_month)

        print(f"{year}-{month}: Collected {len(collected_this_month)} patents")

    print(f"✔️ {year}: Total collected = {len(year_results)} patents")
    all_patents.append(year_results)

1984-01: Collected 142 patents
1984-02: Collected 142 patents
1984-03: Collected 142 patents
1984-04: Collected 142 patents
1984-05: Collected 142 patents
1984-06: Collected 142 patents
1984-07: Collected 142 patents
1984-08: Collected 142 patents
1984-09: Collected 142 patents
1984-10: Collected 142 patents
1984-11: Collected 142 patents
1984-12: Collected 152 patents
✔️ 1984: Total collected = 1714 patents
1985-01: Collected 144 patents
1985-02: Collected 144 patents
1985-03: Collected 144 patents
1985-04: Collected 144 patents
1985-05: Collected 144 patents
1985-06: Collected 144 patents
1985-07: Collected 144 patents
1985-08: Collected 144 patents
1985-09: Collected 144 patents
1985-10: Collected 144 patents
1985-11: Collected 144 patents
1985-12: Collected 148 patents
✔️ 1985: Total collected = 1732 patents
1986-01: Collected 145 patents
1986-02: Collected 145 patents
1986-03: Collected 145 patents
1986-04: Collected 145 patents
1986-05: Collected 145 patents
1986-06: Collected 14

: 

In [1]:
import csv
csv_filename = "patents_foster_1984-2021v"

with open(csv_filename, mode="w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    
    # Write header
    writer.writerow(["year", "country", "doc_number"])
    
    # Flatten the data while writing
    for i in range (len(all_patents)):
        year = 1984 + i
        patent_list = all_patents[i]

        for patent in patent_list:
            writer.writerow([year, patent["country"], patent["doc_number"]])

print(f"CSV file '{csv_filename}' has been created successfully.")

NameError: name 'all_patents' is not defined